In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook

In [27]:
# 망함 네이버 뉴스는 검색건수 상관없이 4000건만 보여줌
keyword="일본불매운동"
start_date='2015.05.01'
end_date='2015.09.30'
# page = 2
start_date_1=re.sub('\.','',start_date)
end_date_1=re.sub('\.','',end_date)
# print(start_date_1)

url_main = 'https://search.naver.com/search.naver?&where=news&query={keyword}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={sd}&de={ed}&docid=&nso=so:r,p:from{sd_1}to{ed_1},a:all&mynews=0&cluster_rank=39&start=1&refresh_start=0'.format(keyword=keyword,sd=start_date,ed=end_date,sd_1=start_date_1,ed_1=end_date_1)
resp_main = requests.get(url_main)
soup_main = BeautifulSoup(resp_main.text, 'lxml')
page_num = soup_main.select_one('#main_pack > div.news.mynews.section._prs_nws > div.section_head > div.title_desc.all_my > span').get_text()
# print(page_num)
page_num = page_num.split(' / ')[1]
print(page_num)
page_num = re.sub(',','',page_num)
page_num = re.sub('건','',page_num)
# print(page_num)
page = int(page_num)//10 + 1
# print(page)
Title=[]
Contents=[]
Date=[]
Press=[]

news_all=pd.DataFrame()
# for i in list(range(1,page)):
for i in tqdm_notebook(list(range(1,page))):
    i = i* 10
    url = 'https://search.naver.com/search.naver?&where=news&query={keyword}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={sd}&de={ed}&docid=&nso=so:r,p:from{sd_1}to{ed_1},a:all&mynews=0&cluster_rank=39&start={index}&refresh_start=0'.format(keyword=keyword,sd=start_date,ed=end_date,index=i,sd_1=start_date_1,ed_1=end_date_1)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'lxml')
    news_list = soup.select('li > dl > dd.txt_inline > a') # 뉴스 url
    for j in list(range(len(news_list))):
        news_url = soup.select('li > dl > dd.txt_inline > a')[j]['href'] # 뉴스 url
        news_resp = requests.get(news_url)
        news_soup = BeautifulSoup(news_resp.text, 'lxml')
        title = news_soup.title.get_text().strip()
        title = title.split(': 네이버')[0]
#         print(news_url)
        contents = news_soup.select_one('#articleBodyContents')
        if contents:
            Title.append(title)
            press = news_soup.select_one('.nclicks\(atp_press\) > img:nth-child(1)')['title']
#             print(press)
            Press.append(press)
            contents = contents.get_text().strip()
#             print(type(contents))
            contents = re.sub("flash 오류를 우회하기 위한 함수 추가\\nfunction _flash_removeCallback\(\) \{\}\\n\\n",'',contents).strip()
            Contents.append(contents)
            date=news_soup.select_one('#main_content > div.article_header > div.article_info > div > span.t11')
            date=date.get_text().split(' ')[0]
#             print(date)
            Date.append(date)
#     print('{}P {}개완료'.format(i//10,len(news_list)), end='\t')
print('완료') 
#     print(resp)

# Title
# Contents

1,109건


완료


* 뉴스 주소
* 정규표현식을 이용해 긁어보자

In [23]:
news_all = pd.DataFrame({'제목':Title,
                        '내용':Contents,
                         '언론사':Press,
                        '날짜':Date})
news_all.head()

,제목,내용,언론사,날짜
0,광주기독교계 미쓰비시 불매운동 동참,// \t\n\t광주지역 기독교계가 일본 기업 미쓰비시 불매운동에 동참했다. 광주기...,경향신문,2015.08.12.
1,롯데 불매운동 속 신동주 일본어 인터뷰 눈길 '부글부글',// 롯데 불매운동[스포츠서울] 신격호 신동빈 부자 전쟁에 소비자들이 롯데 불매운동...,스포츠서울,2015.08.04.
2,"시민단체, ""롯데는 일본기업, 연말까지 불매운동 전개""",// 가족간 경영권 분쟁으로 내홍을 겪고 있는 롯데그룹에 대한 국민 반감이 커지...,파이낸셜뉴스,2015.08.04.
3,[롯데 경영권 분쟁]“실상은 일본기업”… 롯데 불매운동 확산,// ㆍ‘광윤사’가 그룹 좌지우지ㆍ신동주 ‘일본어 인터뷰’도 대중 ‘반일 정서’ 자...,경향신문,2015.08.03.
4,[롯데] 제품 불매운동에 직원들 '억울',// \t\n\t【 앵커멘트 】 롯데그룹의 국적 논란이 제품 불매운동으로 이어지자...,MBN,2015.08.07.


In [24]:
len(news_all)

579

In [25]:
news_all.to_excel('news_2015.xlsx')